In [44]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
import os
import folium

def geomToGeoJSON(in_geom, name, simplify_tolerance, in_ref, out_ref):
    '''
    Matry's Function! Converts Geometry to GeoJSON
    '''
    in_geom = in_geom.Simplify(simplify_tolerance)
    transform = osr.CoordinateTransformation(in_ref, out_ref)      

    #don't want to affect original geometry
    transform_geom = in_geom.Clone()      

    #trasnsform geometry from whatever the local projection is to wgs84

    transform_geom.Transform(transform)
    json_text = transform_geom.ExportToJson()
    #add some attributes

    geom_json = json.loads(json_text)
    #get area in local units

    area = in_geom.GetArea()
    print('area:',round(area*0.00000038610,2))
    geojson_dict = {

        "type": "Feature",

        "geometry": geom_json,

        "properties": {

            "area": area

        }

    }



    geojson = json.dumps(geojson_dict)
    if OUTPUT_GEOJSON:

        f = open('./' + name + '.geojson','w')

        f.write(geojson)

        f.close()

        print('Exported geojson:', name)       

    return geojson

def isolateNetwork(folderPath,siteLayerName,lineLayerName,x,y):
    # Load Lines
    path = folderPath + "\\" + lineLayerName + "\\" + lineLayerName + ".shp"
    
    path_sites = folderPath + "\\" + siteLayerName + "\\" + siteLayerName + ".shp"
    # Buffer around userClick
    print("Now buffer around userClick")
    oRef = osr.SpatialReference()
    oRef.ImportFromEPSG(4326)

    # Reproject
    targRef = osr.SpatialReference()
    targRef.ImportFromEPSG(26918) # NY State Projection
    
    # Create point from x,y and targRef
    ctran = osr.CoordinateTransformation(oRef,targRef)
    [p_lng,p_lat,z] = ctran.TransformPoint(x,y)
    inputPointProj = ogr.Geometry(ogr.wkbPoint)
    inputPointProj.SetPoint_2D(0,p_lng,p_lat)


    print("I got to here")

    geomBuffer = inputPointProj.Buffer(10000) # Buffer 10 meters around the geometry
 
    # Load Selected Sites
    sitesDataSource = ogr.Open(path_sites)
    sl = sitesDataSource.GetLayer()
    ctran = osr.CoordinateTransformation(sl.GetSpatialRef(),targRef)
    
    # Intersect of BUFFER and SITES
    interSites = []
    for site in sl:
        ingeom_site = site.GetGeometryRef()
        ingeom_site.Transform(ctran)
        if ingeom_site.Intersect(geomBuffer):
            # This site is inside the buffer!
            interSites.append((site,ingeom_site))

    # IF #INTER_SITES < 2 (expand circle)?
    print(len(interSites))
    if len(interSites) < 2:
        print("UUUUUUh")
   
    # Load Selected Lines
    dataSource = ogr.Open(path)
    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    linesLayer = dataSource.GetLayer()
    ctranl = osr.CoordinateTransformation(linesLayer.GetSpatialRef(),targRef)
    
    interLines = []
    # Intersect of BUFFER and LINES
    for line in linesLayer:
        ingeom_line = line.GetGeometryRef()
        ingeom_line.Transform(ctranl)
        if ingeom_line.Intersect(geomBuffer):
            interLines.append((line,ingeom_line))
    # Split INTER_LINES based on points (INTER_SITES)
    resLines = []    
    for sentry in interSites:
        s_geom = sentry[1]
        for lentry in interLines:
            l_geom = lentry[1]
            if s_geom.Intersect(l_geom):
                # For testing reasons we will only add the resulting geometry to the list
                resLines.append(s_geom.Intersection(l_geom))
    
    # Navigate Network; Return Network ...
    print(resLines)
    
    # Display all in folium GeoJSON
    m = folium.Map(location=[y,x],zoom_start=13)
    folium.Marker([y,x],popup='<b>USER_CLICK</b>').add_to(m)
    targWebRef = osr.SpatialReference()
    targWebRef.ImportFromEPSG(4326)
    cTransf = osr.CoordinateTransformation(sl.GetSpatialRef(),targWebRef)
    for sentry in interSites:
        
        s_geom = sentry[1] 
       # s_geom.Transform(cTransf)
        print("Adding marker to {0:.10f},{1:.10f}".format(s_geom.GetY(),s_geom.GetX()))
        folium.Marker([s_geom.GetY(),s_geom.GetX()],popup='<b>SITE</b>').add_to(m)
    display(m)
    
    

In [45]:
isolateNetwork("C:\\Users\\mpanozzo\\Desktop\\GDAL Data","SitesSnapped","NHDFlowline",-73.939645996,42.378012068)

Now buffer around userClick
I got to here
13
[]
Adding marker to 4702251.3327124016,587866.0993686076
Adding marker to 4686959.5473698517,594756.6960994259
Adding marker to 4689240.2369169584,594584.8992261988
Adding marker to 4691040.1358349053,594385.3183898183
Adding marker to 4689878.4621657105,595720.6960152539
Adding marker to 4690698.8013147591,596432.6823749561
Adding marker to 4695944.4039908117,578793.4859358763
Adding marker to 4694990.5896716602,579091.7487090918
Adding marker to 4690093.0993314954,578185.1034308940
Adding marker to 4685870.3622813625,581278.9496106951
Adding marker to 4684255.8378062490,582158.2886032270
Adding marker to 4690014.5932377297,589922.7952735049
Adding marker to 4687059.8417991344,589540.5100681020
